In [6]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

EPISODES = 1000

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size) 
        # add another condition to use neural net or random forest or lgbm
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


env = gym.make('SpaceInvaders-v4')
state_size = env.observation_space.shape[0] #377
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
# agent.load("./save/cartpole-dqn.h5")
#done = False
#batch_size = 32

#for e in range(EPISODES):
    # state = env.reset()
#    state = np.reshape(state, [1, state_size]) # call autoencoder on state
#    for time in range(500):
#        env.render()
#        action = agent.act(state)
#        next_state, reward, done, _ = env.step(action)
#        reward = reward if not done else -10
#        next_state = np.reshape(next_state, [1, state_size])
#        agent.remember(state, action, reward, next_state, done)
#        state = next_state
#        if done:
#            print("episode: {}/{}, score: {}, e: {:.2}"
#                  .format(e, EPISODES, time, agent.epsilon))
#            break
#        if len(agent.memory) > batch_size:
#            agent.replay(batch_size)

In [4]:
import gym

In [5]:
env = gym.make('Cart-v4')

In [1]:
import gym
env = gym.make('SpaceInvaders-v4')

In [12]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [30]:
from keras.models import load_model
model = load_model('/media/andres/Baymax/predictor.h5')
encoder = load_model('/media/andres/Baymax/encoder100.h5')

/home/andres/thesis/keras/lib/python3.6/site-packages/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
env = gym.make('SpaceInvaders-v4')
state_size = env.observation_space.shape[0] #377
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)

NameError: name 'DQNAgent' is not defined

In [13]:
state = env.reset()

In [26]:
import matplotlib.pyplot as plt

from skimage import io, color
from skimage.transform import rescale
from skimage.draw import rectangle
from skimage.measure import label, regionprops
from skimage.filters import threshold_otsu
from skimage.morphology import closing, square
import h5py

def load_transform(img):
    img = img[25:195,0:160]
    img_gray = color.rgb2gray(img)
    
    thresh = threshold_otsu(img_gray)
    bw = closing(img_gray > thresh, square(3))
    cleared = bw
    label_image = label(cleared)
    for region in regionprops(label_image):
        if region.area >= 10:
            minr, minc, maxr, maxc = region.bbox
            rr, cc = rectangle(start=(minr, minc), end=(maxr, maxc), shape=cleared.shape)
            cleared[rr, cc] = 1

    image_rescaled = rescale(cleared, 1.0 / 3.0, anti_aliasing=False)
    
    return image_rescaled.reshape(3021)

In [33]:
import numpy as np

state_transformed = load_transform(state)
encoded = encoder.predict(np.expand_dims(state_transformed, axis=0))
prediction = model.predict(encoded)

/home/andres/thesis/keras/lib/python3.6/site-packages/skimage/transform/_warps.py:24: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '
/home/andres/thesis/keras/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [36]:
prediction.argmax()

1

In [28]:
state_transformed.shape

(3021,)